# Descargar datos y dependencias

In [1]:
!wget https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0 -O books.csv
!wget https://www.dropbox.com/s/zpnnoy1i8ljf9fg/goodreads_bert_embeddings.npy?dl=0 -O goodreads_bert_embeddings.npy
!wget https://www.dropbox.com/s/a8hcc9w30y7r3jl/goodreads_bert_large_embeddings.npy?dl=0 -O goodreads_bert_large_embeddings.npy
!wget https://www.dropbox.com/s/dqeqpsr0vdvmcy0/goodreads_past_interactions.json?dl=0 -O goodreads_past_interactions.json
!wget https://www.dropbox.com/s/rjtzhmb2zbpp30q/goodreads_test_interactions.json?dl=0 -O goodreads_test_interactions.json

--2025-07-02 17:39:40--  https://www.dropbox.com/s/57tel5zqopkssrh/books.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0 [following]
--2025-07-02 17:39:40--  https://www.dropbox.com/scl/fi/5s6xrfnu17yi34sfhmskb/books.csv?rlkey=ymzokbyqw3qq2bq5okfao9w1z&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2fef979fb622ae084056072237.dl.dropboxusercontent.com/cd/0/inline/Csu-4GrLvzfyxZzxsZVqpjq_1a0Sxw2qU5-C1Ir3zpH8CkI16fMBwROTxuC_1wHxbFGLmb7iozIwZ_0HxdodTCe8_Ux4t4Xcpdlu-0vsnghlbS5x9UvhwmJSv4gkKsssyT12brR_bhAHr2eaQIsha6Bc/file# [following]
--2025-07-02 17:39:41--  https://uc2fef979fb622ae084056072237.dl.dropboxusercontent.com/cd/0/

In [2]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831126 sha256=725f380a213a30b2e2b8d814985be7302ac2039ee219a0b926367dcdadb8ba17
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [3]:
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
import numpy as np
import pandas as pd
import json
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score, average_precision_score
from collections import Counter


# Cargar Datos

In [4]:
df_books = pd.read_csv('books.csv', sep=',')
df_books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,book_desc
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,Winning will make you famous. Losing means cer...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,Harry Potter's life is miserable. His parents ...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,About three things I was absolutely positive.F...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,The unforgettable novel of a childhood in a sl...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,Alternate Cover Edition ISBN: 0743273567 (ISBN...


In [5]:
# dict index 2 book id and vice-versa for recommendation
idx2bookid = {i: id_ for i, id_ in enumerate(df_books.book_id)}
bookid2idx = {id_:i for i, id_ in enumerate(df_books.book_id)}

In [6]:
with open("goodreads_past_interactions.json", "r") as f:
    user_interactions = json.load(f)

In [7]:
with open("goodreads_test_interactions.json", "r") as f:
    user_interactions_test = json.load(f)

In [8]:
idx2userid = {i: id_ for i, id_ in enumerate(user_interactions.keys())}
userid2idx = {id_:i for i, id_ in enumerate(user_interactions.keys())}

## Manipulación datos

In [9]:
from scipy.sparse import coo_matrix
rows = []
cols = []
data = []

for user_id, book_ids in user_interactions.items():
    for book_id in book_ids:
        u_id = userid2idx[user_id]
        b_id = bookid2idx[book_id]
        rows.append(int(u_id))
        cols.append(int(b_id))
        data.append(1)

max_user_id = max(rows)
max_item_id = max(cols)

# Crear matriz sparse
user_item_matrix_train = coo_matrix((data, (rows, cols)), shape=(max_user_id + 1, max_item_id + 1))
print(user_item_matrix_train.shape)

rows_test = []
cols_test = []
data_test = []

for user_id, book_ids in user_interactions_test.items():
    for book_id in book_ids:
        u_id = userid2idx[user_id]
        b_id = bookid2idx[book_id]
        rows_test.append(int(u_id))
        cols_test.append(int(b_id))
        data_test.append(1)

max_user_id = max([max_user_id] + rows_test)
max_item_id = max([max_item_id] + cols_test)

# Crear matriz sparse
user_item_matrix_test = coo_matrix((data_test, (rows_test, cols_test)), shape=(max_user_id + 1, max_item_id + 1))
print(user_item_matrix_test.shape)

(52821, 4287)
(52821, 4287)


In [10]:
train_mat = user_item_matrix_train.tocsr()
test_mat  = user_item_matrix_test.tocsr()

## Métricas adicionales

In [11]:
num_users = len(userid2idx)
num_items = len(bookid2idx)

print(f"Number of distinct users: {num_users}")
print(f"Number of distinct books: {num_items}")

Number of distinct users: 52821
Number of distinct books: 4287


In [12]:
from collections import Counter
all_train_pairs = [
    (u_idx, i_idx)
    for u_idx, books in user_interactions.items()
    for i_idx in [bookid2idx[b] for b in books]
]
pop_counter = Counter(i for _, i in all_train_pairs)
max_pop = max(pop_counter.values())

In [13]:
# Intra list diversity
def ild_at_k(topk, feat_matrix, k):
    vecs = feat_matrix[topk[:k]]              # (k, D)
    sims = cosine_similarity(vecs)            # (k, k)
    i, j = np.triu_indices(k, k=1)
    return np.mean(1.0 - sims[i, j])

# Novelty basado en popularidad global
def novelty_popularity(topk, pop_counter, max_pop, k):
    pops = np.array([pop_counter[i] for i in topk[:k]], dtype=np.float32)
    return np.mean(1.0 - pops / max_pop)

In [14]:
test_users = [
    userid2idx[u_raw]
    for u_raw in user_interactions_test
    if u_raw in userid2idx
]

In [15]:
bert_embeddings = np.load("goodreads_bert_embeddings.npy")

# Modelo lightFM y entrenamiento

In [16]:
# 5. Entrenar modelo LightFM
model = LightFM(loss='warp')  # o 'bpr', 'warp-kos', 'logistic'
model.fit(user_item_matrix_train, epochs=20, num_threads=8, verbose=True)

Epoch: 100%|██████████| 20/20 [01:07<00:00,  3.35s/it]


In [17]:
# 6. Evaluar
print("Precision@10 (test):", precision_at_k(model, user_item_matrix_test, k=10).mean())
print("AUC (test):", auc_score(model, user_item_matrix_test).mean())

Precision@10 (test): 0.045
AUC (test): 0.9409942


In [18]:
print("Precision@5 (test):", precision_at_k(model, user_item_matrix_test, k=5).mean())
print("Precision@10 (test):", precision_at_k(model, user_item_matrix_test, k=10).mean())
print("Precision@20 (test):", precision_at_k(model, user_item_matrix_test, k=20).mean())

Precision@5 (test): 0.045999993
Precision@10 (test): 0.045
Precision@20 (test): 0.04100001


In [19]:
Ks = [5, 10, 20]
results = {K: {'ndcg':[], 'ap':[], 'ild':[], 'nov':[]} for K in Ks}

for u in test_users:
    # 1) Build candidate set (exclude train positives)
    train_items = set(train_mat[u].indices)
    candidates = np.setdiff1d(
        np.arange(num_items),
        list(train_items),
        assume_unique=True
    )

    # 2) Build the true relevance set for this user
    true_set = set(bookid2idx[b] for b in user_interactions_test[idx2userid[u]])
    # skip if none of their held‐out books are in candidates
    #if not true_set.intersection(candidates):
      #  continue
    # skip if no true positives appear in candidates
    if not (true_set & set(candidates)):
        continue

    # 3) Score all candidates
    u_batch = np.full(len(candidates), u, dtype=np.int32)
    scores  = model.predict(u_batch, candidates).astype(np.float32)

    # 4) Iterate over each K
    for K in Ks:
        # top-K indices & items
        idx = np.argpartition(-scores, K-1)[:K]
        topk = candidates[idx][np.argsort(-scores[idx])]

        # a) full relevance vector for NDCG
        full_rels = np.isin(candidates, list(true_set)).astype(int)
        ndcg_val = ndcg_score([full_rels], [scores], k=K)

        # b) top-K relevance & scores for AP@K
        topk_rels   = np.isin(topk, list(true_set)).astype(int)
        topk_scores = scores[idx][np.argsort(-scores[idx])]
        ap_val      = average_precision_score(topk_rels, topk_scores)

        # c) ILD@K
        ild_val = ild_at_k(topk, bert_embeddings, K)

        # d) Novelty@K
        nov_val = novelty_popularity(topk, pop_counter, max_pop, K)

        # append
        results[K]['ndcg'].append(ndcg_val)
        results[K]['ap'].append(ap_val)
        results[K]['ild'].append(ild_val)
        results[K]['nov'].append(nov_val)

# Finally, print the means
for K in Ks:
    print(f"== Metrics @ {K} ==")
    print(f"Mean NDCG@{K}: {np.mean(results[K]['ndcg']):.4f}")
    print(f"Mean AP@{K}:   {np.mean(results[K]['ap']):.4f}")
    print(f"Mean ILD@{K}:  {np.mean(results[K]['ild']):.4f}")
    print(f"Mean Nov@{K}:  {np.mean(results[K]['nov']):.4f}\n")

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

== Metrics @ 5 ==
Mean NDCG@5: 0.1523
Mean AP@5:   0.2783
Mean ILD@5:  0.3209
Mean Nov@5:  0.4974

== Metrics @ 10 ==
Mean NDCG@10: 0.1472
Mean AP@10:   0.2793
Mean ILD@10:  0.3115
Mean Nov@10:  0.5543

== Metrics @ 20 ==
Mean NDCG@20: 0.1946
Mean AP@20:   0.2608
Mean ILD@20:  0.3080
Mean Nov@20:  0.6269



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


## BERT Embeddings with PCA

In [20]:
bert_embeddings = np.load("goodreads_bert_embeddings.npy")  # shape: (num_items, embedding_dim)
#bert_embeddings_large = np.load("goodreads_bert_large_embeddings.npy")
num_items, embedding_dim = bert_embeddings.shape
#num_items, embedding_dim_large = bert_embeddings.shape
bert_embeddings = bert_embeddings.astype(np.float32)
#bert_embeddings_large = bert_embeddings_large.astype(np.float32)
print(bert_embeddings.shape)
#print(bert_embeddings_large.shape)

(4287, 768)


In [21]:
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

bert_embeddings = normalize(bert_embeddings, norm='l2')

In [22]:
book_id_set = set()

for user_id in user_interactions.keys():
  ids = set(user_interactions[user_id])
  book_id_set.update(ids)

print(len(book_id_set))

4287


Usamos PCA porque si no se demora demasiado.

In [23]:
pca_bert_featmat = PCA(n_components=40).fit_transform(bert_embeddings)

item_features_sparse = csr_matrix(bert_embeddings)
item_features_pca_sparse = csr_matrix(pca_bert_featmat)
# concatenate the identity matrix with the PCA features
item_features_sparse = csr_matrix(np.hstack((np.eye(num_items), item_features_sparse.toarray())))
item_features_pca_sparse = csr_matrix(np.hstack((np.eye(num_items), item_features_pca_sparse.toarray())))
print(item_features_sparse.shape)
print(item_features_pca_sparse.shape)
print(num_items+40)

(4287, 5055)
(4287, 4327)
4327


In [24]:
model_bert = LightFM(loss='warp')
model_bert.fit(interactions=user_item_matrix_train, item_features=item_features_pca_sparse, epochs=20, num_threads=4, verbose=True)

Epoch: 100%|██████████| 20/20 [08:21<00:00, 25.05s/it]


Se me demoró como 2.5 minutos con PCA.

In [25]:
# 6. Evaluar
print("Precision@10 (test):", precision_at_k(model_bert, user_item_matrix_test, item_features=item_features_pca_sparse, k=10).mean())
print("AUC (test):", auc_score(model_bert, user_item_matrix_test, item_features=item_features_pca_sparse).mean())

Precision@10 (test): 0.044
AUC (test): 0.939384


In [26]:
print("Precision@5 (test):", precision_at_k(model_bert, user_item_matrix_test, item_features=item_features_pca_sparse, k=5).mean())
print("Precision@10 (test):", precision_at_k(model_bert, user_item_matrix_test, item_features=item_features_pca_sparse, k=10).mean())
print("Precision@20 (test):", precision_at_k(model_bert, user_item_matrix_test, item_features=item_features_pca_sparse, k=20).mean())

Precision@5 (test): 0.044
Precision@10 (test): 0.044
Precision@20 (test): 0.042000003


In [27]:
Ks = [5, 10, 20]
results = {K: {'ndcg':[], 'ap':[], 'ild':[], 'nov':[]} for K in Ks}

for u in test_users:
    # 1) Build candidate set (exclude train positives)
    train_items = set(train_mat[u].indices)
    candidates = np.setdiff1d(
        np.arange(num_items),
        list(train_items),
        assume_unique=True
    )

    # 2) Build the true relevance set for this user
    true_set = set(bookid2idx[b] for b in user_interactions_test[idx2userid[u]])
    # skip if none of their held‐out books are in candidates
    #if not true_set.intersection(candidates):
      #  continue
    # skip if no true positives appear in candidates
    if not (true_set & set(candidates)):
        continue

    # 3) Score all candidates
    u_batch = np.full(len(candidates), u, dtype=np.int32)
    scores  = model_bert.predict(u_batch, candidates).astype(np.float32)

    # 4) Iterate over each K
    for K in Ks:
        # top-K indices & items
        idx = np.argpartition(-scores, K-1)[:K]
        topk = candidates[idx][np.argsort(-scores[idx])]

        # a) full relevance vector for NDCG
        full_rels = np.isin(candidates, list(true_set)).astype(int)
        ndcg_val = ndcg_score([full_rels], [scores], k=K)

        # b) top-K relevance & scores for AP@K
        topk_rels   = np.isin(topk, list(true_set)).astype(int)
        topk_scores = scores[idx][np.argsort(-scores[idx])]
        ap_val      = average_precision_score(topk_rels, topk_scores)

        # c) ILD@K
        ild_val = ild_at_k(topk, bert_embeddings, K)

        # d) Novelty@K
        nov_val = novelty_popularity(topk, pop_counter, max_pop, K)

        # append
        results[K]['ndcg'].append(ndcg_val)
        results[K]['ap'].append(ap_val)
        results[K]['ild'].append(ild_val)
        results[K]['nov'].append(nov_val)

# Finally, print the means
for K in Ks:
    print(f"== Metrics @ {K} ==")
    print(f"Mean NDCG@{K}: {np.mean(results[K]['ndcg']):.4f}")
    print(f"Mean AP@{K}:   {np.mean(results[K]['ap']):.4f}")
    print(f"Mean ILD@{K}:  {np.mean(results[K]['ild']):.4f}")
    print(f"Mean Nov@{K}:  {np.mean(results[K]['nov']):.4f}\n")

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

== Metrics @ 5 ==
Mean NDCG@5: 0.1109
Mean AP@5:   0.2226
Mean ILD@5:  0.3110
Mean Nov@5:  0.4633

== Metrics @ 10 ==
Mean NDCG@10: 0.0936
Mean AP@10:   0.2155
Mean ILD@10:  0.3255
Mean Nov@10:  0.5175

== Metrics @ 20 ==
Mean NDCG@20: 0.1268
Mean AP@20:   0.2034
Mean ILD@20:  0.3257
Mean Nov@20:  0.5794



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

## ResNet with PCA

In [28]:
portrait_embeddings = np.load("resnet_book_embeddings.npy")  # (num_items, D)

In [29]:
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA

portrait_embeddings = normalize(portrait_embeddings, norm='l2')

In [30]:
pca_resnet_featmat = PCA(n_components=40).fit_transform(portrait_embeddings)


item_features_resnet_pca_sparse = csr_matrix(pca_resnet_featmat)
# concatenate the identity matrix with the PCA features
item_features_resnet_pca_sparse = csr_matrix(np.hstack((np.eye(num_items), item_features_resnet_pca_sparse.toarray())))

print(item_features_resnet_pca_sparse.shape)

(4287, 4327)


In [31]:
model_resnet = LightFM(loss='warp')
model_resnet.fit(interactions=user_item_matrix_train, item_features=item_features_resnet_pca_sparse, epochs=20, num_threads=4, verbose=True)

Epoch: 100%|██████████| 20/20 [08:16<00:00, 24.85s/it]


In [32]:
# 6. Evaluar
print("Precision@10 (test):", precision_at_k(model_resnet, user_item_matrix_test, item_features=item_features_resnet_pca_sparse, k=10).mean())
print("AUC (test):", auc_score(model_resnet, user_item_matrix_test, item_features=item_features_resnet_pca_sparse).mean())

Precision@10 (test): 0.049000002
AUC (test): 0.9398052


In [33]:
print("Precision@5 (test):", precision_at_k(model_resnet, user_item_matrix_test, item_features=item_features_resnet_pca_sparse, k=5).mean())
print("Precision@10 (test):", precision_at_k(model_resnet, user_item_matrix_test, item_features=item_features_resnet_pca_sparse, k=10).mean())
print("Precision@20 (test):", precision_at_k(model_resnet, user_item_matrix_test, item_features=item_features_resnet_pca_sparse, k=20).mean())

Precision@5 (test): 0.055999994
Precision@10 (test): 0.049000002
Precision@20 (test): 0.043000005


In [34]:
Ks = [5, 10, 20]
results = {K: {'ndcg':[], 'ap':[], 'ild':[], 'nov':[]} for K in Ks}

for u in test_users:
    # 1) Build candidate set (exclude train positives)
    train_items = set(train_mat[u].indices)
    candidates = np.setdiff1d(
        np.arange(num_items),
        list(train_items),
        assume_unique=True
    )

    # 2) Build the true relevance set for this user
    true_set = set(bookid2idx[b] for b in user_interactions_test[idx2userid[u]])
    # skip if none of their held‐out books are in candidates
    #if not true_set.intersection(candidates):
      #  continue
    # skip if no true positives appear in candidates
    if not (true_set & set(candidates)):
        continue

    # 3) Score all candidates
    u_batch = np.full(len(candidates), u, dtype=np.int32)
    scores  = model_resnet.predict(u_batch, candidates).astype(np.float32)

    # 4) Iterate over each K
    for K in Ks:
        # top-K indices & items
        idx = np.argpartition(-scores, K-1)[:K]
        topk = candidates[idx][np.argsort(-scores[idx])]

        # a) full relevance vector for NDCG
        full_rels = np.isin(candidates, list(true_set)).astype(int)
        ndcg_val = ndcg_score([full_rels], [scores], k=K)

        # b) top-K relevance & scores for AP@K
        topk_rels   = np.isin(topk, list(true_set)).astype(int)
        topk_scores = scores[idx][np.argsort(-scores[idx])]
        ap_val      = average_precision_score(topk_rels, topk_scores)

        # c) ILD@K
        ild_val = ild_at_k(topk, bert_embeddings, K)

        # d) Novelty@K
        nov_val = novelty_popularity(topk, pop_counter, max_pop, K)

        # append
        results[K]['ndcg'].append(ndcg_val)
        results[K]['ap'].append(ap_val)
        results[K]['ild'].append(ild_val)
        results[K]['nov'].append(nov_val)

# Finally, print the means
for K in Ks:
    print(f"== Metrics @ {K} ==")
    print(f"Mean NDCG@{K}: {np.mean(results[K]['ndcg']):.4f}")
    print(f"Mean AP@{K}:   {np.mean(results[K]['ap']):.4f}")
    print(f"Mean ILD@{K}:  {np.mean(results[K]['ild']):.4f}")
    print(f"Mean Nov@{K}:  {np.mean(results[K]['nov']):.4f}\n")

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

== Metrics @ 5 ==
Mean NDCG@5: 0.1145
Mean AP@5:   0.2276
Mean ILD@5:  0.2988
Mean Nov@5:  0.5214

== Metrics @ 10 ==
Mean NDCG@10: 0.1029
Mean AP@10:   0.2451
Mean ILD@10:  0.2975
Mean Nov@10:  0.5724

== Metrics @ 20 ==
Mean NDCG@20: 0.1472
Mean AP@20:   0.2141
Mean ILD@20:  0.3039
Mean Nov@20:  0.6359



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_ranking.py:1033: UserWarning: No positive c

In [35]:
def custom_ranking_metrics_at_k(rec_func, train_user_items, test_user_items, K=10, show_progress=False):

    # Ensure matrices are in CSR format
    train_user_items = train_user_items.tocsr()
    test_user_items = test_user_items.tocsr()

    num_users, num_items = test_user_items.shape
    relevant = 0
    total_precision_div = 0
    total_map = 0
    total_ndcg = 0
    total_auc = 0
    total_users = 0

    # Compute cumulative gain for NDCG normalization
    cg = 1.0 / np.log2(np.arange(2, K + 2))  # Discount factor
    cg_sum = np.cumsum(cg)  # Ideal DCG normalization

    # Get users with at least one item in the test set
    users_with_test_data = np.where(np.diff(test_user_items.indptr) > 0)[0]

    # Progress bar
    #progress = tqdm.tqdm(total=len(users_with_test_data), disable=not show_progress)

    batch_size = 1000
    start_idx = 0

    while start_idx < len(users_with_test_data):
        batch_users = users_with_test_data[start_idx:start_idx + batch_size]
        recommended_items = rec_func(batch_users, N=K)
        start_idx += batch_size

        for user_idx, user_id in enumerate(batch_users):
            test_items = set(test_user_items.indices[test_user_items.indptr[user_id]:test_user_items.indptr[user_id + 1]])

            if not test_items:
                continue  # Skip users without test data

            num_relevant = len(test_items)
            total_precision_div += min(K, num_relevant)

            ap = 0
            hit_count = 0
            auc = 0
            idcg = cg_sum[min(K, num_relevant) - 1]  # Ideal Discounted Cumulative Gain (IDCG)
            num_negative = num_items - num_relevant

            for rank, item in enumerate(recommended_items[user_idx]):
                if item in test_items:
                    relevant += 1
                    hit_count += 1
                    ap += hit_count / (rank + 1)
                    total_ndcg += cg[rank] / idcg
                else:
                    auc += hit_count  # Accumulate hits for AUC calculation

            auc += ((hit_count + num_relevant) / 2.0) * (num_negative - (K - hit_count))
            total_map += ap / min(K, num_relevant)
            total_auc += auc / (num_relevant * num_negative)
            total_users += 1

        #progress.update(len(batch_users))

    #progress.close()

    # Compute final metrics
    precision = relevant / total_precision_div if total_precision_div > 0 else 0
    mean_ap = total_map / total_users if total_users > 0 else 0
    mean_ndcg = total_ndcg / total_users if total_users > 0 else 0
    mean_auc = total_auc / total_users if total_users > 0 else 0

    return {
        "precision": precision,
        "map": mean_ap,
        "ndcg": mean_ndcg,
        "auc": mean_auc
    }

In [36]:
# define a recommendation function for the lightfm model
def rec_function(users, N=10):
    recommendations = []
    for user in users:
        uids = [user]* num_items
        scores = model_bert.predict(uids, np.arange(num_items), item_features=item_features_pca_sparse)
        top_items = np.argsort(-scores)[:N]
        recommendations.append(top_items)
    return np.array(recommendations)

# define a recommendation function for the lightfm model
def rec_function_2(users, N=10):
    recommendations = []
    all_items = np.arange(num_items)
    for user in users:
        past_interactions = user_interactions[idx2userid[user]]
        past_interactions = np.array([bookid2idx[i] for i in past_interactions])
        available_items = np.setdiff1d(all_items, past_interactions)
        uids = [user]* len(available_items)
        scores = model_bert.predict(uids, available_items, item_features=item_features_pca_sparse)
        top_items = np.argsort(-scores)[:N]
        recommendations.append(top_items)
    return np.array(recommendations)

In [37]:
print(custom_ranking_metrics_at_k(rec_function, user_item_matrix_train, user_item_matrix_test, K=10))
print(custom_ranking_metrics_at_k(rec_function_2, user_item_matrix_train, user_item_matrix_test, K=10))

{'precision': 0.044, 'map': 0.013757539682539682, 'ndcg': np.float64(0.04430935135087546), 'auc': 0.5208842646714986}
{'precision': 0.025, 'map': 0.006938492063492063, 'ndcg': np.float64(0.023688562156345055), 'auc': 0.5113607668926815}
